<a href="https://colab.research.google.com/github/joaopaulof19/banks_project/blob/main/banks_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Laboratório prático: aquisição e processamento de informações sobre os maiores bancos do mundo

**Cenário do Projeto**
<br>
Uma empresa multinacional contratou você como engenheiro de dados. Seu trabalho é acessar e processar dados de acordo com os requisitos.

Seu chefe pediu que você compilasse a lista dos 10 maiores bancos do mundo, classificados por capitalização de mercado em bilhões de dólares. Além disso, você precisa transformar os dados e armazená-los em USD, GBP, EUR e INR de acordo com as informações da taxa de câmbio disponibilizadas como um arquivo CSV. Você deve salvar a tabela de informações processadas localmente em formato CSV e como uma tabela de banco de dados. Gestores de diferentes países consultarão a tabela do banco de dados para extrair a lista e anotar o valor da capitalização de mercado em sua própria moeda.

**instruções**

    1. Escreva uma função para extrair as informações tabulares do URL fornecido sob o título By Market Capitalization e salve-as em um quadro de dados.
    2. Escreva uma função para transformar o quadro de dados adicionando colunas para Capitalização de Mercado em GBP , EUR e INR , arredondadas para 2 casas decimais, com base nas informações de taxa de câmbio compartilhadas como um arquivo CSV.
    3. Escreva uma função para carregar o quadro de dados transformado em um arquivo CSV de saída.
    4. Escreva uma função para carregar o quadro de dados transformado em um servidor de banco de dados SQL como uma tabela.
    5. Escreva uma função para executar consultas na tabela do banco de dados.
    6. Execute as seguintes consultas na tabela do banco de dados:
    a. Extraia as informações do escritório de Londres, ou seja, Nome e MC_GBP_Billion
    b. Extraia as informações do escritório de Berlim, ou seja, Nome e MC_EUR_Billion
    c. Extraia as informações do escritório de Nova Delhi, ou seja, Nome e MC_INR_Billion
    7. Escreva uma função para registrar o progresso do código.
    8. Ao executar os comandos de inicialização de dados e chamadas de função, mantenha as entradas de log apropriadas.

In [1]:
!pip install requests
!pip install bs4
!pip install pandas
!pip install sqlite3
!pip install numpy

ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


In [2]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv

--2024-02-21 01:29:42--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45 [text/csv]
Saving to: ‘exchange_rate.csv’

exchange_rate.csv   100%[===================>]      45  --.-KB/s    in 0s      

2024-02-21 01:29:43 (13.5 MB/s) - ‘exchange_rate.csv’ saved [45/45]



In [3]:
import pandas as pd
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import numpy as np
import sqlite3

In [4]:
def log_progress(message):
  timestamp_format = '%Y-%h-%d-%H:%M:%S'
  now = datetime.now()
  timestamp = now.strftime(timestamp_format)
  print(timestamp + ' : ' + message + '\n')
  with open(log, "a") as f:
    f.write(timestamp + ' : ' + message + '\n')


In [5]:
def extract(url, table_attribs):
    ''' This function aims to extract the required
    information from the website and save it to a data frame. The
    function returns the data frame for further processing. '''

    # Fazendo a requisição GET para obter o conteúdo da página
    response = requests.get(url)

    # Criando o objeto BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    # Encontrando todas as tabelas com os dados desejados
    # tabelas = soup.find_all('tbody')
    tabelas = soup.find_all('table')


    # Selecionando a segunda tabela (índice 1, pois a contagem começa em 0)
    tabela_desejada = tabelas[0]

    # Inicializando listas para armazenar os dados
    data = {"Rank": [], "Bank name": [], "Market Cap": []}

    # Iterando sobre as linhas da tabela, ignorando o cabeçalho
    for linha in tabela_desejada.find_all('tr')[1:]:
        colunas = linha.find_all('td')
        # Verificando se a linha tem o número esperado de colunas
        if len(colunas) == 3:
            data["Rank"].append(colunas[0].text.strip())
            data["Bank name"].append(colunas[1].text.strip())
            data["Market Cap"].append(colunas[2].text.strip())

    # Criando o DataFrame
    df = pd.DataFrame(data)

    log_progress('Extração de dados concluída. Iniciando o processo de transformação ')
    return df

In [6]:
def transform(df, csv_path):
    ''' This function accesses the CSV file for exchange rate
    information, and adds three columns to the data frame, each
    containing the transformed version of Market Cap column to
    respective currencies'''
    cambio = pd.read_csv(csv_path)
    exchange_rate = cambio.set_index('Currency').to_dict()['Rate']
    df['Market Cap'] = df['Market Cap'].astype('float64')
    df.rename(columns={'Market Cap': 'MC_USD_Billion'}, inplace=True)

    df['MC_GBP_Billion'] = [np.round(x*exchange_rate['GBP'],2) for x in df['MC_USD_Billion']]
    df['MC_EUR_Billion'] = [np.round(x*exchange_rate['EUR'],2) for x in df['MC_USD_Billion']]
    df['MC_INR_Billion'] = [np.round(x*exchange_rate['INR'],2) for x in df['MC_USD_Billion']]

    log_progress('Transformação de dados concluída. Iniciando o processo de carregamento')

    return df

In [7]:
def load_to_csv(df, output_path):
    ''' This function saves the final data frame as a CSV file in
    the provided path. Function returns nothing.'''
    df.to_csv(output_path)
    log_progress('Dados salvos em arquivo CSV ')

In [8]:
def load_to_db(df, sql_connection, table_name):
    ''' This function saves the final data frame to a database
    table with the provided name. Function returns nothing.'''
    df.to_sql(name=table_name, con=sql_connection, if_exists="replace", index=False)
    log_progress('Dados carregados no banco de dados como uma tabela, executando consultas')

In [9]:
def run_query(query_statement, sql_connection):
    ''' This function runs the query on the database table and
    prints the output on the terminal. Function returns nothing. '''
    print(query_statement)
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_output)
    log_progress('Processo concluído')


In [10]:
''' Here, you define the required entities and call the relevant
functions in the correct order to complete the project. Note that this
portion is not inside any function.'''

' Here, you define the required entities and call the relevant\nfunctions in the correct order to complete the project. Note that this\nportion is not inside any function.'

In [11]:
url = "https://web.archive.org/web/20230908091635 /https://en.wikipedia.org/wiki/List_of_largest_banks"
csv = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv "
atributos = ["Name", " MC_USD_Billion"]
# atributos_finais = ["Name", " MC_USD_Billion", " MC_GBP_Billion", " MC_EUR_Billion", "MC_INR_Billion"]
file_csv = "./Largest_banks_data.csv"
db = 'Banks.db'
log = "code_log.txt"
table_attribs = ["Rank", "Bank name", "Market Cap"]


In [12]:
df = extract(url, table_attribs)

2024-Feb-21-01:29:45 : Extração de dados concluída. Iniciando o processo de transformação 



In [13]:
df

,Rank,Bank name,Market Cap
0,1,JPMorgan Chase,432.92
1,2,Bank of America,231.52
2,3,Industrial and Commercial Bank of China,194.56
3,4,Agricultural Bank of China,160.68
4,5,HDFC Bank,157.91
5,6,Wells Fargo,155.87
6,7,HSBC Holdings PLC,148.90
7,8,Morgan Stanley,140.83
8,9,China Construction Bank,139.82
9,10,Bank of China,136.81


In [14]:
df.dtypes

Rank          object
Bank name     object
Market Cap    object
dtype: object

In [15]:
df= transform(df=df, csv_path=csv)


2024-Feb-21-01:29:46 : Transformação de dados concluída. Iniciando o processo de carregamento



In [16]:
df

,Rank,Bank name,MC_USD_Billion,MC_GBP_Billion,MC_EUR_Billion,MC_INR_Billion
0,1,JPMorgan Chase,432.92,346.34,402.62,35910.71
1,2,Bank of America,231.52,185.22,215.31,19204.58
2,3,Industrial and Commercial Bank of China,194.56,155.65,180.94,16138.75
3,4,Agricultural Bank of China,160.68,128.54,149.43,13328.41
4,5,HDFC Bank,157.91,126.33,146.86,13098.63
5,6,Wells Fargo,155.87,124.70,144.96,12929.42
6,7,HSBC Holdings PLC,148.90,119.12,138.48,12351.26
7,8,Morgan Stanley,140.83,112.66,130.97,11681.85
8,9,China Construction Bank,139.82,111.86,130.03,11598.07
9,10,Bank of China,136.81,109.45,127.23,11348.39


In [17]:
df['MC_EUR_Billion'][4]

146.86

In [18]:

print(df.loc[df['Rank'] == '5', 'MC_EUR_Billion'])


4    146.86
Name: MC_EUR_Billion, dtype: float64


In [19]:

load_to_csv(df, output_path=file_csv)


2024-Feb-21-01:29:46 : Dados salvos em arquivo CSV 



In [20]:
conn = sqlite3.connect(db)
log_progress('Conexão SQL iniciada')
load_to_db(df, conn, 'Largest_banks')


2024-Feb-21-01:29:46 : Conexão SQL iniciada

2024-Feb-21-01:29:46 : Dados carregados no banco de dados como uma tabela, executando consultas



In [21]:
query_statement = "SELECT * FROM Largest_banks"
run_query(query_statement, sql_connection= conn)

SELECT * FROM Largest_banks
  Rank                                Bank name  MC_USD_Billion  \
0    1                           JPMorgan Chase          432.92   
1    2                          Bank of America          231.52   
2    3  Industrial and Commercial Bank of China          194.56   
3    4               Agricultural Bank of China          160.68   
4    5                                HDFC Bank          157.91   
5    6                              Wells Fargo          155.87   
6    7                        HSBC Holdings PLC          148.90   
7    8                           Morgan Stanley          140.83   
8    9                  China Construction Bank          139.82   
9   10                            Bank of China          136.81   

   MC_GBP_Billion  MC_EUR_Billion  MC_INR_Billion  
0          346.34          402.62        35910.71  
1          185.22          215.31        19204.58  
2          155.65          180.94        16138.75  
3          128.54         

In [22]:
query_statement = "SELECT AVG(MC_GBP_Billion) FROM Largest_banks"
run_query(query_statement, sql_connection= conn)

SELECT AVG(MC_GBP_Billion) FROM Largest_banks
   AVG(MC_GBP_Billion)
0              151.987
2024-Feb-21-01:29:46 : Processo concluído



In [23]:
query_statement = 'SELECT "Bank name" from Largest_banks LIMIT 5'
run_query(query_statement, sql_connection= conn)

SELECT "Bank name" from Largest_banks LIMIT 5
                                 Bank name
0                           JPMorgan Chase
1                          Bank of America
2  Industrial and Commercial Bank of China
3               Agricultural Bank of China
4                                HDFC Bank
2024-Feb-21-01:29:46 : Processo concluído



In [24]:
conn.close()